## CS431/631 Data Intensive Distributed Analytics
### Winter 2021 - Assignment 3
---

**Please edit this (text) cell to provide your name and UW student ID number!**
* **Name:** Shahzeb Naveed
* **ID:** 20789222

---
#### Overview
For this assignment, you will be using Python and Spark to perform some graph analysis, using a graph of the Gnutella server network.   In this graph, each node represents a server, and each (directed) edge represents a connection between servers in Gnutella's peer-to-peer network.  The input file for this assignment, `p2p-Gnutella08-adj.txt`, represents the graph as an adjacency list.   Each server (node) is identified by a unique number, and each line in the file gives the adjacency list for a single server.
For example, this line:
> 91	243	1923	2194

gives the adjacency list for server `91`.   It indicates that there are edges from server `91` to servers `243`, `1923`, and `2194`.    According to the Stanford Network Analysis Project, which collected these data, [the graph includes 6301 servers and 20777 edges](http://snap.stanford.edu/data/p2p-Gnutella08.html).

Run the following block to install Spark and download the input file.

In [1]:
!apt-get update -qq > /dev/null
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://downloads.apache.org/spark/spark-2.4.7/spark-2.4.7-bin-hadoop2.7.tgz
!tar xf spark-2.4.7-bin-hadoop2.7.tgz
!pip install -q findspark
!wget -q https://student.cs.uwaterloo.ca/~cs451/content/cs431/p2p-Gnutella08-adj.txt

and then create a `SparkContext`:

In [2]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.7-bin-hadoop2.7"

import findspark
findspark.init()

from pyspark import SparkContext, SparkConf

spark_conf = SparkConf()\
  .setAppName("YourTest")\
  .setMaster("local[*]")

sc = SparkContext.getOrCreate(spark_conf)

---
### Important

###### The questions that follow ask you to implement functions whose prototypes are given to you. Do **NOT** change the prototypes of the functions. Do **NOT** write code outside of functions. All necessary code should be included in the function body (except for import statements). You may declare functions inside of the function body. When marking, we will execute your code by calling the functions from an external program, which is why your code cannot rely on statements running outside functions. Please remove any call to the functions that you may have introduced for test purposes before submitting your notebook.

---
#### Question 1  (6/24 marks):

To get warmed up, you should first write Spark code to confirm or determine some basic properties of the Gnutella graph.  Write code in the provided functions that will return answers to the following questions, as specified in each function's documentation:
- How many nodes and edges are there in the graph?  (This should confirm the numbers given above.)
- How many nodes of each outdegree are there? That is, how many nodes have no outgoing edges, how many have one outgoing edge, how many have two outgoing edges, and so on?
- How many nodes of each indegree are there?

You should use Spark to answer these questions.   Do *not* load the entire graph into your Python driver program.

In [3]:
def num_nodes_edges():
    """Returns a tuple (num_nodes, num_edges)"""
    #### Your code for Question 1.1 should go here

    def tokenize_tabs(s):
      return s.split('\t')

    file = "p2p-Gnutella08-adj.txt"
    text_file = sc.textFile(file)
    text_file.cache()

    # Tokenize and organize; cache() for performance because data is reused
    data = text_file.map(lambda x: tokenize_tabs(x)).map(lambda x: (x[0], x[1:]))
    data.cache()

    # Count nodes
    num_nodes = data.count()

    # Count edges
    num_edges = data.map(lambda x: (x[0], len(x[1]))).values().sum()

    return (num_nodes, num_edges)
    
    
def out_counts():
    """Returns a dictionary where the keys are the outdegrees, and the 
    values are the number of nodes of the corresponding outdegree """
    #### Your code for Question 1.2 should go here

    def tokenize_tabs(s):
      return s.split('\t')

    file = "p2p-Gnutella08-adj.txt"
    text_file = sc.textFile(file)
    text_file.cache()

    # Tokenize and organize
    data = text_file.map(lambda x: tokenize_tabs(x)).map(lambda x: (x[0], x[1:]))

    # Count out-degree
    outdegree_dict = data.map(lambda x: (len(x[1]), x[0])).groupByKey().mapValues(len).sortByKey().collect()

    return dict(outdegree_dict)
    

def in_counts():
    """Returns a dictionary where the keys are the indegrees, and the 
    values are the number of nodes of the corresponding indegree """
    #### Your code for Question 1.3 should go here

    def tokenize_tabs(s):
      return s.split('\t')

    file = "p2p-Gnutella08-adj.txt"
    text_file = sc.textFile(file)
    text_file.cache()

    # Tokenize and organize
    data = text_file.map(lambda x: tokenize_tabs(x)).map(lambda x: (x[0], x[1:]))

    # Count in_degree for every key
    in_degree = data.flatMap(lambda x: x[1]).map(lambda x: (str(x), 1)).reduceByKey(lambda x,y: x+y)
    
    # Groupby in_degree
    in_degree_dict = in_degree.map(lambda x: (x[1], x[0])).groupByKey().mapValues(len).sortByKey().collect()
    
    return dict(in_degree_dict)

---
Your main objective for this assignment is to perform *single source personalized page rank* over the Gnutella graph.  To get started, you should make sure that you have a clear understanding of ordinary (i.e., non-personalized) page rank.  Read the description of page rank in Section 5.3 of [the course textbook](http://mapreduce.cc/).   Personalized page rank is like ordinary page rank except:
- One node in the graph is designated as the *source* node. Personalized page rank is performed with respect to that source node.
- Personalized page rank is initialized by assigning all probability mass to the source node, and none to the other nodes. In contrast, ordinary page rank is initialized by giving all nodes the same probability mass.
- Whenever personalized page rank makes a random jump, it jumps back to the source node. In contrast, ordinary page rank may jump to any node.
- In personalized page rank, all probability mass lost dangling nodes is put back into the source nodes.  In ordinary page rank, lost mass is distributed evenly over all nodes.

#### Question 2  (10/24 marks):

Your task is to write a Spark program to perform personalized page rank over the Gnutella graph for a specified number of iterations, and of course a specific node. The function you will implement takes three input values:
- source node id (a non-negative integer)
- iteration count (a positive integer)
- random jump factor value (a float between 0 and 1) - This is 1-B as introduced in the lecture.

The function should perform personalized page rank, with respect to the specified source node, over the Gnutella graph, for the specified number of iterations, using Spark.
The output of your function should be a list of the 10 nodes with the highest personalized page rank with respect to the given source. For each of the 10 nodes, return the node's id and page rank value as a tuple. The list returned by the function should therefore look something like this: `[(node_id_1, highest_pagerank_value), ..., (node_id_10, 10th_highest_pagerank_value)]`

In [4]:
def personalized_page_rank(source_node_id, num_iterations, jump_factor):
    """Returns a list of the 10 nodes with the highest page rank value along with their value, as tuples
    [(node_id_1, highest_pagerank_value), ..., (node_id_10, 10th_highest_pagerank_value)]"""
    # your solution to Question 2 here

    def tokenize_tabs(s):
      return s.split('\t')

    def compute_contribs(page_rank, adjacency_list, source_node_id, jump_factor):
      
      if len(adjacency_list)!=0:
        
        # If a node has neighbours, contribute probability mass to them
        for m in adjacency_list:
          yield (m, ((1 - jump_factor) * page_rank) / len(adjacency_list))

        # Jumps only to source node
        yield (source_node_id, jump_factor * page_rank)

      else:
        # If a node has no neighbours, contribute to source node
        yield (source_node_id, page_rank)

    def convert_str_to_int_list(str_list):
      return list(map(int, str_list))
      
    # Read file
    file = "p2p-Gnutella08-adj.txt"
    text_file = sc.textFile(file)

    # Get graph structure
    graph = text_file.map(lambda x: tokenize_tabs(x)).map(lambda x: (int(x[0]), convert_str_to_int_list(x[1:])))
    graph.cache()

    # Init probabilities
    page_ranks = graph.map(lambda x: (x[0], 1) if x[0] == source_node_id else ((x[0], 0)))

    # Keep updating probabilities
    for i in range(num_iterations):
      graph_with_ranks = graph.join(page_ranks)
      scores = graph_with_ranks.flatMap(lambda x: compute_contribs(page_rank=x[1][1], adjacency_list=x[1][0], source_node_id=source_node_id, jump_factor=jump_factor))
      page_ranks = scores.reduceByKey(lambda x,y: x+y)

    sorted = page_ranks.map(lambda x: (x[1], x[0])).sortByKey(False).map(lambda x: (x[1], x[0]))

    return sorted.take(10)

---
#### Question 3  (4/24 marks):

For the previous question, you implemented personalized page rank that ran for a specified number of iterations.  However, it is also common to write iterative algorithms that run until some specified termination condition is reached.
For example, for page rank, suppose the $p_i(x)$ represents the probability mass assigned to node $x$ after the $i$th iteration of the algorithm.  ($p_0(x)$ is the initial probability mass of node $x$.)   We define the change of $x$'s probability mass on the $i$th iteration as $\lvert p_i(x)-p_{i-1}(x) \rvert$.   Then, we can iterate personalized page rank until the maximum (over all nodes) change is less than a specified threshold, i.e, until all nodes' page ranks have converged.

For this question, modify your personalized page rank implementation from Question 2 so that it iterates until the 
maximum node change is less than $\frac{0.5}{N}$, where $N$ represents the number of nodes in the graph.
This version of the function should take only two inputs: the source node id and the random jump factor.

In [6]:
def personalized_page_rank_stopping_criterion(source_node_id, jump_factor):
    """Returns a list of the 10 nodes with the highest page rank value along with their value, as tuples
    [(node_id_1, highest_pagerank_value), ..., (node_id_10, 10th_highest_pagerank_value)]"""
    # your solution to Question 3 here

    def tokenize_tabs(s):
      return s.split('\t')

    def compute_contribs(page_rank, adjacency_list, source_node_id, jump_factor):
      
      if len(adjacency_list)!=0:
        
        # If a node has neighbours, contribute probability mass to them
        for m in adjacency_list:
          yield (m, ((1 - jump_factor) * page_rank) / len(adjacency_list))

        # Jumps only to source node
        yield (source_node_id, jump_factor * page_rank)

      else:
        # If a node has no neighbours, contribute to source node
        yield (source_node_id, page_rank)

    def convert_str_to_int_list(str_list):
      return list(map(int, str_list))
      
    # Read file
    file = "p2p-Gnutella08-adj.txt"
    text_file = sc.textFile(file)
    text_file.cache() # cache or else it will be computed twice

    # Get graph structure
    graph = text_file.map(lambda x: tokenize_tabs(x)).map(lambda x: (int(x[0]), convert_str_to_int_list(x[1:])))
    graph.cache()

    # Get number of nodes and calculate threshold
    nodes = text_file.count()
    threshold = 0.5 / nodes

    # Init probabilities
    page_ranks = graph.map(lambda x: (x[0], 1) if x[0] == source_node_id else ((x[0], 0)))
    
    # Keep updating page ranks until they have converged
    while(True):
      graph_with_ranks = graph.join(page_ranks)
      scores = graph_with_ranks.flatMap(lambda x: compute_contribs(page_rank=x[1][1], adjacency_list=x[1][0], source_node_id=source_node_id, jump_factor=jump_factor))
      page_ranks_new = scores.reduceByKey(lambda x,y: x+y)
      page_ranks_comparison = page_ranks.join(page_ranks_new).map(lambda x: (x[0], x[1][0], x[1][1], abs(x[1][1] - x[1][0])))
      page_ranks = page_ranks_new
      max_change = page_ranks_comparison.map(lambda x: (x[0], x[3])).values().max()
      
      if max_change < threshold:
        break

    sorted = page_ranks.map(lambda x: (x[1], x[0])).sortByKey(False).map(lambda x: (x[1], x[0]))

    return sorted.take(10)

---
#### Question 4  (4/24 marks):

Spark provides the ability to *cache* RDDs.   This is useful when an RDD will be used more than once.   Instead of computing the same RDD multiple times, it can be computed once, cached, and then re-used from the cache.   Read about caching in the Spark textbook, or in the [persistence section of the Spark RDD programming guide](https://spark.apache.org/docs/latest/rdd-programming-guide.html#rdd-persistence).   Caching can be particularly effective for iterative Spark applications, like those you are writing for this assignment.

For this question, go back to the code that you wrote to answer Question 3, and add caching.   (Caching will not affect the functionality of your code, i.e., what it computes.   It should only affect performance.)   Don't worry about different persistence levels.   Just use `cache()`, which caches RDDs at the default persistence level.

In addition to adding `cache()` calls in your code, use the text cell below to briefly explain how you decided which RDDs to cache. 

If you were not able to finish Question 3, add caching annotations to your solution for Question 2 instead.

---
#### Your answer to Question 4:

I cached two RDDs. The `graph` rdd because it has to be resued in every loop so if it's cached, it does not have to be recomputed every time. The `text_file` rdd is also cached because in Question 3, it will otherwise be computed twice, once while calculating threshold and once when the `page_ranks_new` is computed for the first time (first time only because graph is cached and we won't be needing the `text_file` in subsequent runs of the loop. 


---
Don't forget to save your workbook!   When you are finished and you are ready to submit your assignment, download your notebook file (.ipynb) from the hub to your machine, and then follow the submission instructions in the assignment.